<a href="https://colab.research.google.com/github/m11dedhia/sdes/blob/master/SDES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
def p10(arr):
  return [arr[2], arr[4], arr[1], arr[6], arr[3], arr[9], arr[0], arr[8], arr[7], arr[5]]

def p8(arr):
  return [arr[5], arr[2], arr[6], arr[3], arr[7], arr[4], arr[9], arr[8]]

def p4(arr):
  return [arr[1], arr[3], arr[2], arr[0]]

def left_shift_1bit(arr):
  return [*arr[1:], arr[0]]

def left_shift_2bit(arr):
  return [*arr[2:], *arr[:2]]

def cut_half(arr):
  return arr[:len(arr) // 2], arr[len(arr) // 2:]

def ip8(arr):
  return [arr[1], arr[5], arr[2], arr[0], arr[3], arr[7], arr[4], arr[6]]

def ip8_inverse(arr):
  return [arr[3], arr[0], arr[2], arr[4], arr[6], arr[1], arr[7], arr[5]]

def ep8(arr):
  return [arr[3], arr[0], arr[1], arr[2], arr[1], arr[2], arr[3], arr[0]]

s0 = np.array([[[0, 1], [0, 0], [1, 1], [1, 0]],
               [[1, 1], [1, 0], [0, 1], [0, 0]],
               [[0, 0], [1, 0], [0, 1], [1, 1]],
               [[1, 1], [0, 1], [1, 1], [1, 0]]])

s1 = np.array([[[0, 0], [0, 1], [1, 0], [1, 1]],
               [[1, 0], [0, 0], [0, 1], [1, 1]],
               [[1, 1], [0, 0], [0, 1], [0, 0]],
               [[1, 0], [0, 1], [0, 0], [1, 1]]])

In [ ]:
def generate_keys(key):
  half1, half2 = cut_half(p10(key))
  left_shift1_half1 = left_shift_1bit(half1)
  left_shift1_half2 = left_shift_1bit(half2)
  left_shift2_half1 = left_shift_2bit(left_shift1_half1)
  left_shift2_half2 = left_shift_2bit(left_shift1_half2)
  key1 = p8(np.concatenate((left_shift1_half1, left_shift1_half2)))
  key2 = p8(np.concatenate((left_shift2_half1, left_shift2_half2)))
  return key1, key2

In [ ]:
def fiestel(left_half, right_half, key):
  expanded_right = ep8(right_half)
  xored = np.bitwise_xor(expanded_right, key)
  xored_left_half, xored_right_half = cut_half(xored)
  p_s_boxed = p4(np.concatenate((s0[(2 * xored_left_half[0]) + (xored_left_half[3])][(2 * xored_left_half[1]) + (xored_left_half[2])], s1[(2 * xored_right_half[0]) + (xored_right_half[3])][(2 * xored_right_half[1]) + (xored_right_half[2])])))
  return np.bitwise_xor(left_half, p_s_boxed), right_half

In [ ]:
def encrypt(plain_text, key):
  key1, key2 = generate_keys(key)
  left_half, right_half = cut_half(ip8(plain_text))
  before_switch_left, before_switch_right = fiestel(left_half, right_half, key1)
  # after switch
  after_switch_left, after_switch_right = fiestel(right_half, before_switch_left, key2)
  cipher = ip8_inverse(np.concatenate((after_switch_left, after_switch_right)))
  return cipher

In [ ]:
print(encrypt([0, 1, 0, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 0, 1, 1, 1, 0]))

[1, 0, 1, 1, 0, 0, 0, 0]


In [ ]:
def decrypt(cipher_text, key):
  key1, key2 = generate_keys(key)
  left_half, right_half = cut_half(ip8(cipher_text))
  before_switch_left, before_switch_right = fiestel(left_half, right_half, key2)
  # after switch
  after_switch_left, after_switch_right = fiestel(right_half, before_switch_left, key1)
  plain = ip8_inverse(np.concatenate((after_switch_left, after_switch_right)))
  return plain

In [ ]:
print(decrypt([1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 1, 1, 1, 0]))

[0, 1, 0, 1, 1, 1, 0, 1]
